<a href="https://colab.research.google.com/github/fjadidi2001/AD_Prediction/blob/main/MRtoCT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

> The Falah/Alzheimer_MRI Disease Classification dataset is a valuable resource for researchers and health medicine applications. This dataset focuses on the classification of Alzheimer's disease based on MRI scans. The dataset consists of brain MRI images labeled into four categories:

- '0': Mild_Demented
- '1': Moderate_Demented
- '2': Non_Demented
- '3': Very_Mild_Demented



In [ ]:
from datasets import load_dataset

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Print the number of examples
print("Number of examples:", len(dataset))

# Print the structure of the first example
print("Structure of the first example:")
print(dataset[0])

# Print the keys of the first example (if it's a dictionary)
if isinstance(dataset[0], dict):
    print("Keys in the first example:", dataset[0].keys())

# Print the first few samples
print("Sample data:")
for example in dataset[:5]:
    print(example)

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

# Assuming the image is stored under the key 'image' in the dataset
image = dataset[0]['image']

# Display the image
plt.imshow(image)
plt.axis('off')  # Hide the axes
plt.show()

In [ ]:
# Display 25 images with their labels
plt.figure(figsize=(15, 10))  # Set the figure size
for i in range(25):
    # Get the image and label
    image = dataset[i]['image']
    label = dataset[i]['label']

    # Plot the image
    plt.subplot(5, 5, i + 1)  # Arrange images in a grid (2 rows, 5 columns)
    plt.imshow(image)  # Display the image
    plt.title(f"Label: {label}")  # Set the title as the label
    plt.axis('off')  # Hide the axes

plt.tight_layout()  # Adjust layout to avoid overlapping
plt.show()

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Extract labels (assuming the key is 'label')
labels = [int(example['label']) for example in dataset]  # Convert labels to integers

# Count the occurrences of each label
label_counts = Counter(labels)

# Print the label distribution
print("Label distribution:", label_counts)

# Set a better color palette
sns.set_palette("muted")

# Plot the distribution
plt.figure(figsize=(10, 6))
sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()), palette="viridis",hue=list(label_counts.keys()))

# Add labels and title
plt.xlabel('Labels', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Distribution of Labels in the Train Dataset before augmentation', fontsize=16, fontweight='bold')

# Add grid lines for better readability
plt.grid(True, axis='y', linestyle='-', alpha=0.7)

# Customize x-axis to show only integer labels
plt.xticks(list(label_counts.keys()), fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='test')

# Extract labels (assuming the key is 'label')
labels = [int(example['label']) for example in dataset]  # Convert labels to integers

# Count the occurrences of each label
label_counts = Counter(labels)

# Print the label distribution
print("Label distribution:", label_counts)

# Set a better color palette
sns.set_palette("pastel")  # You can choose other palettes like "deep", "muted", "bright", etc.

# Plot the distribution
plt.figure(figsize=(10, 6))
sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()), palette="viridis",hue=list(label_counts.keys()))

# Add labels and title
plt.xlabel('Labels', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Distribution of Labels in the Test Dataset before augmentation', fontsize=16, fontweight='bold')

# Add grid lines for better readability
plt.grid(True, axis='y', linestyle='-', alpha=0.7)

# Customize x-axis to show only integer labels
plt.xticks(list(label_counts.keys()), fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

> Data augmentation is a powerful technique to increase the diversity of your training dataset by applying random transformations such as rotations, flips, zooms, and more. This helps improve the generalization of your model, especially when the dataset is small or imbalanced.



In [ ]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Define augmentation transformations
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,       # Randomly rotate by up to 15 degrees
    width_shift_range=0.1,   # Randomly shift width by up to 10%
    height_shift_range=0.1,  # Randomly shift height by up to 10%
    horizontal_flip=True,    # Randomly flip horizontally
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    zoom_range=0.2,          # Randomly zoom by up to 20%
)

# Function to apply augmentation to an image
def augment_image(image):
    # Convert PIL image to NumPy array if necessary
    if not isinstance(image, np.ndarray):
        image = np.array(image)  # Convert PIL image to NumPy array

    # Ensure the image has 3 channels (RGB)
    if len(image.shape) == 2:  # If grayscale, convert to RGB
        image = np.stack((image,) * 3, axis=-1)  # Shape: (height, width, 3)
    elif len(image.shape) == 3 and image.shape[-1] == 4:  # If RGBA, convert to RGB
        image = image[:, :, :3]  # Keep only the first 3 channels

    # Print the shape of the image before augmentation
    print(f"Shape before augmentation: {image.shape}")

    # Add batch dimension: (1, height, width, channels)
    image = np.expand_dims(image, axis=0)

    # Apply augmentation
    augmented_image = next(datagen.flow(image, batch_size=1))[0]  # Use next() to get the augmented image

    # Print the shape of the image after augmentation
    print(f"Shape after augmentation: {augmented_image.shape}")

    return augmented_image

# Display original and augmented images with labels
plt.figure(figsize=(15, 10))
for i in range(5):  # Show 5 examples
    # Original image
    original_image = dataset[i]['image']  # Replace 'image' with the correct key
    label = dataset[i]['label']  # Replace 'label' with the correct key

    # Check the type of the image
    print(f"Image type: {type(original_image)}")

    # Display original image
    plt.subplot(5, 2, 2 * i + 1)
    plt.imshow(original_image)
    plt.title(f"Original {i+1} - Label: {label}")
    plt.axis('off')

    # Augmented image
    augmented_image = augment_image(original_image)
    plt.subplot(5, 2, 2 * i + 2)
    plt.imshow(augmented_image.astype(np.uint8))  # Ensure the image is in the correct format for display
    plt.title(f"Augmented {i+1} - Label: {label}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageOps

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Define augmentation transformations for grayscale images
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,       # Randomly rotate by up to 15 degrees
    width_shift_range=0.1,   # Randomly shift width by up to 10%
    height_shift_range=0.1,  # Randomly shift height by up to 10%
    horizontal_flip=True,    # Randomly flip horizontally
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    zoom_range=0.2,          # Randomly zoom by up to 20%
)

# Function to convert image to grayscale
def to_grayscale(image):
    if isinstance(image, np.ndarray):
        if len(image.shape) == 3 and image.shape[-1] == 3:  # If RGB, convert to grayscale
            image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    else:  # If PIL image, convert to grayscale
        image = ImageOps.grayscale(image)
    return image

# Function to apply augmentation to a grayscale image
def augment_image(image):
    # Convert image to NumPy array if necessary
    if not isinstance(image, np.ndarray):
        image = np.array(image)  # Convert PIL image to NumPy array

    # Ensure the image has a single channel (grayscale)
    if len(image.shape) == 2:  # If grayscale, add a channel dimension
        image = np.expand_dims(image, axis=-1)  # Shape: (height, width, 1)

    # Add batch dimension: (1, height, width, 1)
    image = np.expand_dims(image, axis=0)

    # Apply augmentation
    augmented_image = next(datagen.flow(image, batch_size=1))[0]  # Use next() to get the augmented image
    return augmented_image

# Display original and augmented grayscale images with labels
plt.figure(figsize=(15, 10))
for i in range(5):  # Show 5 examples
    # Original image
    original_image = dataset[i]['image']  # Replace 'image' with the correct key
    label = dataset[i]['label']  # Replace 'label' with the correct key

    # Convert original image to grayscale
    original_grayscale = to_grayscale(original_image)

    # Display original grayscale image
    plt.subplot(5, 2, 2 * i + 1)
    plt.imshow(original_grayscale, cmap='gray')
    plt.title(f"Original {i+1} - Label: {label}")
    plt.axis('off')

    # Augmented grayscale image
    augmented_image = augment_image(original_grayscale)
    plt.subplot(5, 2, 2 * i + 2)
    plt.imshow(augmented_image.squeeze(), cmap='gray')  # Remove extra dimensions and display
    plt.title(f"Augmented {i+1} - Label: {label}")
    plt.axis('off')

plt.tight_layout()
plt.show()

In [ ]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from PIL import ImageOps

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Define augmentation transformations for grayscale images
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,       # Randomly rotate by up to 15 degrees
    width_shift_range=0.1,   # Randomly shift width by up to 10%
    height_shift_range=0.1,  # Randomly shift height by up to 10%
    horizontal_flip=True,    # Randomly flip horizontally
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    zoom_range=0.2,          # Randomly zoom by up to 20%
)

# Function to convert image to grayscale
def to_grayscale(image):
    if isinstance(image, np.ndarray):
        if len(image.shape) == 3 and image.shape[-1] == 3:  # If RGB, convert to grayscale
            image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    else:  # If PIL image, convert to grayscale
        image = ImageOps.grayscale(image)
    return image

# Function to apply augmentation to a grayscale image
def augment_image(image):
    # Convert image to NumPy array if necessary
    if not isinstance(image, np.ndarray):
        image = np.array(image)  # Convert PIL image to NumPy array

    # Ensure the image has a single channel (grayscale)
    if len(image.shape) == 2:  # If grayscale, add a channel dimension
        image = np.expand_dims(image, axis=-1)  # Shape: (height, width, 1)

    # Add batch dimension: (1, height, width, 1)
    image = np.expand_dims(image, axis=0)

    # Apply augmentation
    augmented_image = next(datagen.flow(image, batch_size=1))[0]  # Use next() to get the augmented image
    return augmented_image

# Augment the dataset and collect labels
augmented_labels = []
for example in dataset:
    original_image = example['image']  # Replace 'image' with the correct key
    label = example['label']  # Replace 'label' with the correct key

    # Convert original image to grayscale
    original_grayscale = to_grayscale(original_image)

    # Augment the grayscale image
    augmented_image = augment_image(original_grayscale)

    # Append the label to the augmented_labels list
    augmented_labels.append(label)

# Count the occurrences of each label in the augmented dataset
augmented_label_counts = Counter(augmented_labels)

# Print the label distribution
print("Augmented label distribution:", augmented_label_counts)

# Plot the distribution
plt.figure(figsize=(10, 6))
plt.bar(augmented_label_counts.keys(), augmented_label_counts.values(), color='skyblue')

# Add labels and title
plt.xlabel('Labels', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Distribution of Labels After Augmentation', fontsize=16, fontweight='bold')

# Customize x-axis to show only integer labels
plt.xticks(list(augmented_label_counts.keys()), fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

> The augmentation process is not modifying the dataset or the labels. This happens because:

1. The augmentation is applied to the images, but the labels remain unchanged.

2. The augmented images are not being saved or used to update the dataset.



### Steps:
- Define Augmentation Ratios:

For each class, specify how many augmented images to generate per original image.

- Apply Augmentation:

For each image in the dataset, generate the specified number of augmented images based on its class.

- Collect Augmented Images and Labels:

Store the augmented images and their corresponding labels.

- Visualize the Distribution:

Plot the distribution of labels in the augmented dataset.

In [ ]:
from datasets import load_dataset
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from PIL import ImageOps

# Load the Falah/Alzheimer_MRI dataset
dataset = load_dataset('Falah/Alzheimer_MRI', split='train')

# Define augmentation transformations for grayscale images
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=15,       # Randomly rotate by up to 15 degrees
    width_shift_range=0.1,   # Randomly shift width by up to 10%
    height_shift_range=0.1,  # Randomly shift height by up to 10%
    horizontal_flip=True,    # Randomly flip horizontally
    brightness_range=[0.8, 1.2],  # Randomly adjust brightness
    zoom_range=0.2,          # Randomly zoom by up to 20%
)

# Function to convert image to grayscale
def to_grayscale(image):
    if isinstance(image, np.ndarray):
        if len(image.shape) == 3 and image.shape[-1] == 3:  # If RGB, convert to grayscale
            image = np.dot(image[..., :3], [0.2989, 0.5870, 0.1140]).astype(np.uint8)
    else:  # If PIL image, convert to grayscale
        image = ImageOps.grayscale(image)
    return image

# Function to apply augmentation to a grayscale image
def augment_image(image):
    # Convert image to NumPy array if necessary
    if not isinstance(image, np.ndarray):
        image = np.array(image)  # Convert PIL image to NumPy array

    # Ensure the image has a single channel (grayscale)
    if len(image.shape) == 2:  # If grayscale, add a channel dimension
        image = np.expand_dims(image, axis=-1)  # Shape: (height, width, 1)

    # Add batch dimension: (1, height, width, 1)
    image = np.expand_dims(image, axis=0)

    # Apply augmentation
    augmented_image = next(datagen.flow(image, batch_size=1))[0]  # Use next() to get the augmented image
    return augmented_image

# Define the number of augmented images to generate per class
augmentation_ratios = {
    0: 7,  # For class 0, generate 7 augmented images per original image
    1: 102,  # For class 1, generate 102 augmented images per original image
    2: 2,  # For class 2, generate 1 augmented image per original image
    3: 3,  # For class 3, generate 3 augmented images per original image
}

# Augment the dataset and collect labels
augmented_images = []
augmented_labels = []

for example in dataset:
    original_image = example['image']  # Replace 'image' with the correct key
    label = example['label']  # Replace 'label' with the correct key

    # Convert original image to grayscale
    original_grayscale = to_grayscale(original_image)

    # Generate augmented images based on the class
    num_augmented = augmentation_ratios.get(label, 1)  # Default to 1 if label not in augmentation_ratios
    for _ in range(num_augmented):
        augmented_image = augment_image(original_grayscale)
        augmented_images.append(augmented_image)
        augmented_labels.append(label)

# Count the occurrences of each label in the augmented dataset
augmented_label_counts = Counter(augmented_labels)

# Print the label distribution
print("Augmented label distribution:", augmented_label_counts)

# Plot the distribution
plt.figure(figsize=(10, 6))
plt.bar(augmented_label_counts.keys(), augmented_label_counts.values(), color='black')

# Add labels and title
plt.xlabel('Labels', fontsize=14)
plt.ylabel('Count', fontsize=14)
plt.title('Distribution of Labels After Augmentation', fontsize=16, fontweight='bold')

# Customize x-axis to show only integer labels
plt.xticks(list(augmented_label_counts.keys()), fontsize=12)

# Show the plot
plt.tight_layout()
plt.show()

In [ ]:
from collections import defaultdict
import random

# Combine original and augmented data
all_images = list(dataset['image']) + augmented_images
all_labels = list(dataset['label']) + augmented_labels

# Separate images by class
class_images = defaultdict(list)
for image, label in zip(all_images, all_labels):
    class_images[label].append(image)

# Randomly sample 10 images from each class
sampled_images = {}
for label, images in class_images.items():
    sampled_images[label] = random.sample(images, min(10, len(images)))  # Sample 10 images or all if less than 10

# Display sampled images
plt.figure(figsize=(20, 20))
for label, images in sampled_images.items():
    for i, image in enumerate(images):
        plt.subplot(4, 10, label * 10 + i + 1)  # Arrange in a grid (4 classes x 10 images)
        plt.imshow(image, cmap='gray')
        plt.title(f"Class {label}")
        plt.axis('off')

plt.tight_layout()
plt.show()

# Apr4

## Step 1: Load and Preprocess the MRI Dataset

